## 네이버 뉴스 크롤링

In [4]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from pprint import pprint
from tqdm.notebook import tqdm

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 전문 가져오기 >_select 사용
- 네이버 뉴스만 가져와서 결과값 조금 작음 
- 결과 메모장 저장 -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
RESULT_PATH = './'
now = datetime.now() #파일이름 현 시간으로 저장하기

def get_news(n_url):
    news_detail = []

    breq = requests.get(n_url, headers={'User-Agent':'Mozilla/5.0'})
    bsoup = BeautifulSoup(breq.content)

    title = bsoup.find('h2', class_='media_end_head_headline').text
    news_detail.append(title)

    pdate = bsoup.find('span', class_="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME").text[:11]
    news_detail.append(pdate)

    _text = bsoup.find('div', class_='newsct_article _article_body').get_text().replace('\n', " ")
    _text = _text.replace("    동영상 뉴스   ", "")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())
  
    news_detail.append(n_url)
    
    pcompany = bsoup.find('div', class_='copyright').find('p', class_='c_text').text
    pcompany = pcompany.split('All rights reserved.')[0].split('Copyright ⓒ')[1].strip()
    news_detail.append(pcompany)

    return news_detail

def crawler(maxpage,query,s_date,e_date):

    crawled_data = {}
    years = []
    company = []
    title = []
    contents = []
    link = []
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    f = open(f"./contents_text_{query}.txt", 'w', encoding='utf-8-sig')
    
    for page in tqdm(range(1, maxpage_t+1, 10)):
    
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        req = requests.get(url,
                          headers={'User-Agent':'Mozilla/5.0'})
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont)
        
        for urls in soup.find_all("a", class_="info"):
            try :
                if urls.get_text() == "네이버뉴스":
                    print(urls["href"])
                    news_detail = get_news(urls["href"])
                        # pdate, pcompany, title, btext
                    f.write("{}\t{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[4], news_detail[0], news_detail[2],news_detail[3]))  # new style
                    years.append(news_detail[1])
                    company.append(news_detail[4])
                    title.append(news_detail[0])
                    contents.append(news_detail[2])
                    link.append(news_detail[3])
                    
            except Exception as e:
                print(e)
                continue
        
    crawled_data["years"] = years
    crawled_data["company"] = company
    crawled_data["title"] = title
    crawled_data["contents"] = contents
    crawled_data["link"] = link
    f.close()
    return crawled_data
    
def excel_make(crawled_data):
    data = pd.DataFrame(crawled_data)
    
    xlsx_outputFileName = f'{query}_({s_date}~{e_date}).xlsx'
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8-sig', index=False)


maxpage = input("최대 출력할 페이지수 입력하시오: ")
if not maxpage.isdigit():
    maxpage = input("최대 출력할 페이지수 입력하시오: ")
query = input("검색어 입력: ")
s_date = input("시작날짜 입력(ex) 2021.01.01):")
e_date = input("끝날짜 입력(ex) 2021.05.24):")
crawled_data = crawler(maxpage,query,s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다. 
excel_make(crawled_data) #엑셀로 만들기

최대 출력할 페이지수 입력하시오: 200
검색어 입력: 애플
시작날짜 입력(ex) 2021.01.01):2022.11.01
끝날짜 입력(ex) 2021.05.24):2022.12.05


  0%|          | 0/200 [00:00<?, ?it/s]

https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1
https://n.news.naver.com/mnews/article/215/0001069625?sid=101
https://n.news.naver.com/mnews/article/081/0003322308?sid=104
https://n.news.naver.com/mnews/article/081/0003322208?sid=104
https://n.news.naver.com/mnews/article/366/0000859856?sid=104
https://n.news.naver.com/mnews/article/366/0000859665?sid=101
https://n.news.naver.com/mnews/article/092/0002275519?sid=105
https://n.news.naver.com/mnews/article/023/0003732640?sid=105
https://n.news.naver.com/mnews/article/008/0004825609?sid=104
https://n.news.naver.com/mnews/article/015/0004783778?sid=101
https://n.news.naver.com/mnews/article/011/0004130152?sid=105
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=11
https://n.news.naver.com/mnews/article/001/0013621417?sid=104
https://n.news.naver.

https://n.news.naver.com/mnews/article/031/0000714056?sid=105
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=131
https://n.news.naver.com/mnews/article/215/0001068388?sid=101
https://n.news.naver.com/mnews/article/092/0002275398?sid=105
https://n.news.naver.com/mnews/article/031/0000714056?sid=105
https://n.news.naver.com/mnews/article/469/0000710218?sid=105
https://n.news.naver.com/mnews/article/003/0011566592?sid=105
https://n.news.naver.com/mnews/article/015/0004782454?sid=104
https://n.news.naver.com/mnews/article/277/0005184177?sid=101
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=141
https://n.news.naver.com/mnews/article/032/0003189925?sid=101
https://n.news.naver.com/mnews/article/421/0006493163?sid=105
https://n.news.naver.com/mnews/article/277/0005184177?sid=101
https://n.news.nav

https://n.news.naver.com/mnews/article/015/0004782372?sid=101
https://n.news.naver.com/mnews/article/469/0000710131?sid=105
https://n.news.naver.com/mnews/article/018/0005376602?sid=101
https://n.news.naver.com/mnews/article/366/0000859129?sid=104
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=271
https://n.news.naver.com/mnews/article/366/0000859129?sid=104
https://n.news.naver.com/mnews/article/016/0002071494?sid=104
https://n.news.naver.com/mnews/article/277/0005184869?sid=101
https://n.news.naver.com/mnews/article/374/0000312473?sid=101
https://n.news.naver.com/mnews/article/119/0002662129?sid=104
https://n.news.naver.com/mnews/article/421/0006490497?sid=104
https://n.news.naver.com/mnews/article/018/0005377414?sid=101
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=281
https://n.news.nav

...

https://n.news.naver.com/mnews/article/023/0003726847?sid=105
https://n.news.naver.com/mnews/article/092/0002273863?sid=105
https://n.news.naver.com/mnews/article/018/0005371134?sid=103
https://n.news.naver.com/mnews/article/023/0003728232?sid=101
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1221
https://n.news.naver.com/mnews/article/023/0003728232?sid=101
https://n.news.naver.com/mnews/article/092/0002273896?sid=105
https://n.news.naver.com/mnews/article/421/0006440523?sid=104
https://n.news.naver.com/mnews/article/018/0005368154?sid=101
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1231
https://n.news.naver.com/mnews/article/018/0005368154?sid=101
https://n.news.naver.com/mnews/article/001/0013555405?sid=104
https://n.news.naver.com/mnews/article/092/0002273284?sid=105
https://n.news.n

https://n.news.naver.com/mnews/article/056/0011368902?sid=104
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1401
https://n.news.naver.com/mnews/article/056/0011368902?sid=104
https://n.news.naver.com/mnews/article/138/0002136714?sid=105
https://n.news.naver.com/mnews/article/076/0003937148?sid=103
https://n.news.naver.com/mnews/article/052/0001810690?sid=104
https://n.news.naver.com/mnews/article/030/0003057517?sid=105
https://n.news.naver.com/mnews/article/001/0013560257?sid=101
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1411
https://n.news.naver.com/mnews/article/468/0000900746?sid=106
'NoneType' object has no attribute 'text'
https://n.news.naver.com/mnews/article/417/0000867547?sid=101
https://n.news.naver.com/mnews/article/018/0005366398?sid=101
https://n.news.naver.com/mnews/artic

https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1641
https://n.news.naver.com/mnews/article/609/0000651587?sid=106
'NoneType' object has no attribute 'text'
https://n.news.naver.com/mnews/article/008/0004817054?sid=101
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1651
https://n.news.naver.com/mnews/article/138/0002136650?sid=105
https://n.news.naver.com/mnews/article/018/0005360442?sid=101
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1661
https://n.news.naver.com/mnews/article/138/0002136114?sid=105
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1671
https://search.naver.com/search.naver?where=

https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1851
https://n.news.naver.com/mnews/article/056/0011386763?sid=103
https://n.news.naver.com/mnews/article/031/0000707627?sid=105
https://n.news.naver.com/mnews/article/047/0002375117?sid=103
https://search.naver.com/search.naver?where=news&query=애플&sort=0&ds=2022.11.01&de=2022.12.05&nso=so%3Ar%2Cp%3Afrom20221101to20221205%2Ca%3A&start=1861
https://n.news.naver.com/mnews/article/052/0001809581?sid=104
https://n.news.naver.com/mnews/article/009/0005038779?sid=102
https://n.news.naver.com/mnews/article/138/0002136044?sid=105
https://n.news.naver.com/mnews/article/374/0000309247?sid=101
https://n.news.naver.com/mnews/article/029/0002763952?sid=104
https://n.news.naver.com/mnews/article/374/0000309048?sid=101
https://n.news.naver.com/mnews/article/119/0002664341?sid=106
'NoneType' object has no attribute 'text'
https://search.naver.com/search.nave

ModuleNotFoundError: No module named 'openpyxl'

In [2]:
import pandas as pd

data = pd.read_excel(f'data/{query}_({s_date}~{e_date}).xlsx')
data

,years,company,title,contents,link
0,2022.09.06.,중앙일보.,"""아이폰14 프로 100달러 올릴것""…삼성 안올렸는데 애플은 왜",애플이 오는 7일(현지시간) 아이폰14 시리즈를 공개할 예정이다. 아이폰14 예상 ...,https://n.news.naver.com/mnews/article/025/000...
1,2022.09.06.,ZDNet Korea.,"LT게임즈 신작 '신선놀음', 구글-애플 마켓 정식 출시",방치형 수련 RPG 장르LT게임즈 글로벌(LTGAMES GLOBAL)은 방치형 수련...,https://n.news.naver.com/mnews/article/092/000...
2,2022.09.06.,매일경제.,[단독] 여당 '애플 갑질 차단법' 세계 최초로 발의했다,수수료 폭리·광고 무임승차與 '비용전가 방지법' 마련◆ 애플 갑질 차단법 ◆ ...,https://n.news.naver.com/mnews/article/009/000...
3,2022.09.05.,연합뉴스.,"'개과천선' 애플 시리 ""한국, 독도에 확고한 영토 주권 행사""","""독도는 누구 땅"" 질문에 엉터리 답변했다가 반크 항의에 혼쭐·시정 애플 시리...",https://n.news.naver.com/mnews/article/001/001...
4,2022.09.06.,이데일리.,"'개발자 채용' 점핏, 애플 앱스토어 '오늘의 앱' 선정",[이데일리 강경래 기자] IT(정보기술) 개발자 채용 플랫폼 점핏은 애플 앱스토어 ...,https://n.news.naver.com/mnews/article/018/000...
...,...,...,...,...,...
5045,2022.04.26.,ZDNet Korea.,"애플, 노조 결성 대응 위해 반노조 로펌 소속 변호사 고용",스타벅스 노조 결성 막아온 리틀러 멘델슨 소속 변호사 고용애플이 애플스토어 노동조합...,https://n.news.naver.com/mnews/article/092/000...
5046,2022.04.26.,ZDNet Korea.,애플 아이폰14 연기설 나와...중국 봉쇄 공급망 차질 탓,현지 반도체 협력사 연기 가능성 제기코로나19 확산으로 봉쇄정책을 실시하고 있는 중...,https://n.news.naver.com/mnews/article/092/000...
5047,2022.04.26.,데일리안.,[단독] 공정위에 납작 엎드린 애플…韓 지원에 돈 ‘2배’ 썼다,공정위에 ‘동의의결 이행 결과’ 첫 제출지난해 R&D지원센터 외 계획대로 이행아이폰...,https://n.news.naver.com/mnews/article/119/000...
5048,2022.05.12.,SBS Biz.,[서학개미 뭐샀나] 애플·테슬라 급락…나스닥100 3배 추종 ETF 관심 지속,■ 경제와이드 모닝벨 '서학 개미 브리핑' - 정다인오늘(12일)도 어김없이 서학개...,https://n.news.naver.com/mnews/article/374/000...
